In [11]:
import os
import openai
import streamlit as st
import pandas as pd

from llama_index import (
    Document,
    GPTVectorStoreIndex,
    LLMPredictor,
    PromptHelper,
    ServiceContext,
    LangchainEmbedding
)

from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index.prompts.prompts import QuestionAnswerPrompt



# APIキーなどの設定
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_type = 'azure'
openai.api_version = '2023-05-15'

In [23]:

def llama_index_generate(references):
    """llama-indexによるインデックスの生成"""
    # LLM Predictor
    llm_predictor = LLMPredictor(llm=AzureChatOpenAI(
        deployment_name='GPT35TURBO',         # デプロイ名
        max_tokens=1000,                      # 最大トークン数
        temperature=0,                        # 出力のランダム度合い
        openai_api_version=openai.api_version # openaiのapiのバージョン情報
    ))

    #テキストの埋め込みベクトル変換(Embedding)に関する設定
    embeddings = LangchainEmbedding(OpenAIEmbeddings(
        engine="ADA",       # エンベディングに使うモデル
        chunk_size=1,                         # ここでのチャンクサイズはバッチサイズ
        openai_api_version=openai.api_version # openaiのapiのバージョン情報
    ))

    #  Prompt Helper（テキスト分割に関する設定）
    prompt_helper = PromptHelper(
        max_input_size=3000,    # 最大入力サイズ
        num_output=1000,        # LLMの出力サイズ
        chunk_size_limit=3000,  # 使用する最大チャンクサイズ（チャンク：テキストを細かく分割したもの）
        max_chunk_overlap=0,    # チャンクオーバーラップの最大トークン数
        separator="。"          # テキスト分割の区切り文字
    )

    # Service Context（インデックスを作ったりクエリを実行する際に必要になるものをまとめたもの）
    service_context = ServiceContext.from_defaults(
        llm_predictor=llm_predictor, # LLM Predictor
        embed_model=embeddings,      # エンベディングについての設定
        prompt_helper=prompt_helper  # Prompt Helper
    )

    # インデックスの生成
    index = GPTVectorStoreIndex.from_documents(
        references,                      # 参考として与えたデータ（商品リスト）
        service_context=service_context, # Service Context
        prompt_helper=prompt_helper      # Prompt Helper
    )

    return index

def llama_index_getdocument(text_list):
    """テキストのリストをまとめたdocumentsを返す"""
    documents = [Document(t) for t in text_list]

    return documents

def llama_generate(index, query, top_k):
    """llama-indexによる回答の生成"""
    # 与えるコンテキスト（商品リストのうちクエリとの類似度が高いもの）をもとに回答をもとめるようなプロンプト
    QA_PROMPT_TMPL = (
        "私たちは以下の情報をコンテキスト情報として与えます。 \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "この情報をもとに質問に日本語で回答してください。: {query_str}\n"
    )
    qa_prompt = QuestionAnswerPrompt(QA_PROMPT_TMPL)
    
    # 回答生成
    with st.spinner("検索中（1分ほどかかります）..."):
        # プロンプトと上位いくつまでの類似度を使用するか設定
        query_engine = index.as_query_engine(
            text_qa_template=qa_prompt, # 上記のプロンプトを与える（デフォルトは英語文）
            similarity_top_k=top_k      # 参考情報（商品リスト）のうちクエリとの類似度上位何件を生成に利用するか
        )
        # 生成
        response = query_engine.query(query)

    return response

def get_chatgpt_response(past_messages):
    """ChatGPTにより回答生成"""
    response = openai.ChatCompletion.create(
        engine='gpt-35-turbo',
        messages=past_messages,
        max_tokens=1500
    )
    return response.choices[0].message.content

In [6]:
!mkdir -p 'data/paul_graham/'
!curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -o 'data/paul_graham/paul_graham_essay.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75042  100 75042    0     0   181k      0 --:--:-- --:--:-- --:--:--  181k


In [18]:
# 商品リストを変換
df = pd.read_csv("list.csv", encoding="shift-jis")
columns = df.columns
df["_text"] = ""
for column in columns:
    df["_text"] = df["_text"] + f"【{column}】" + df[column]
document_list = df["_text"].values
documents = llama_index_getdocument(document_list)



In [19]:
documents

[Document(text='【本の名前】Python入門【本の概要】Pythonの基本的な文法や使い方を解説した入門書【本のターゲット読者】初心者向け【どういった人に読んでもらいたいか】プログラミング初心者やPythonを学びたい人に', doc_id='5da42f18-fc39-4299-8a20-d62337f109d5', embedding=None, doc_hash='a0f6e90fec9a17b25bffd3ceff7712ddebe26ebcae90e37816810ce3ff30c443', extra_info=None),
 Document(text='【本の名前】Effective Java【本の概要】Java開発において実践的なアドバイスを提供する書籍【本のターゲット読者】中級者向け【どういった人に読んでもらいたいか】Java開発に携わるプログラマに', doc_id='2d8ca0b0-b751-4eaa-91ad-aa925554b4b9', embedding=None, doc_hash='a4e4b299c75492284b0a0944796f24c7f1ff3b402ffa0e688e3302bbf5b7a3ca', extra_info=None),
 Document(text='【本の名前】クリーンアーキテクチャ【本の概要】アーキテクチャの重要性や設計原則を解説し、実践的な開発手法を提供する書籍【本のターゲット読者】エンジニア向け【どういった人に読んでもらいたいか】ソフトウェアアーキテクトや開発チームリーダーに', doc_id='2af0f5c1-7c09-409e-bd33-7c6fe38c3b4e', embedding=None, doc_hash='288547b25645f2494c8ed2bb18663f7b99e6dc1d4ffd8d8b318c92037d3b5481', extra_info=None),
 Document(text='【本の名前】経営と人間性【本の概要】ビジネスの世界において、人間的な価値観を重視する姿勢が持つ意義について考察するエッセイ【本のターゲット読者】経営者向け【どういった人に読んでもらいたいか】経営者やビジネスパーソンに', doc_id='26f906fa-a8

In [21]:
# テキストのリストをまとめたdocumentsを返す


index = llama_index_generate(documents)

/opt/conda/lib/python3.11/site-packages/langchain/embeddings/openai.py:214: UserWarning: WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
  warnings.warn(
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easi

In [22]:
index

In [25]:
# クエリ （description：アップロードした顧客情報)
query = f"あなたは顧客に商品を推薦する営業です。\
以下の顧客情報に一番適しているものを提案してください。\
その理由も回答してください。\
また、推薦する際に顧客が購入したくなるような文章を生成してください。\
顧客情報： \
出力形式は以下のようにしてください。\
適しているもの：\
選んだ理由：\
推薦メッセージ："

# llama-indexによる回答の生成
result = llama_generate(index=index, query=query, top_k=1)

# 回答の表示
result = str(result)
print(result)

2024-01-11 00:04:53.632 
  command:

    streamlit run /opt/conda/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


適しているもの：マーケティングの秘訣
選んだ理由：顧客情報によると、この顧客はビジネスの成功に興味があり、ビジネスマンであるとされています。そのため、マーケティングの知識を学ぶことは彼らのビジネスにとって非常に重要であり、彼らの成功に役立つでしょう。

推薦メッセージ：「ビジネスの成功に興味をお持ちの方におすすめの書籍『マーケティングの秘訣』です。この本は、ビジネスにおいて重要なマーケティングの知識を実践的な事例や著者自身の体験を交えて語っています。あなたのビジネスに役立つ情報が満載であり、成功への一歩を踏み出すためのヒントが詰まっています。ぜひ一読してみてください。」
